<table align="left" style="border-style: hidden" class="table"> <tr><td class="col-md-2"><img style="float" src="http://prob140.org/assets/icon_sp22_ugarte.png" alt="Prob140 Logo" style="width: 120px;"/></td><td><div align="left"><h3 style="margin-top: 0;">Probability for Data Science</h3><h4 style="margin-top: 20px;">UC Berkeley, Fall 2022</h4><p>Ani Adhikari</p>CC BY-NC-SA 4.0</div></td></tr></table><!-- not in pdf -->

This content is protected and may not be shared, uploaded, or distributed.

In [ ]:
from datascience import *
from prob140 import *     
import numpy as np
from scipy import special
import pickle

In [ ]:
# Run this code

import itertools as it
from collections import Counter

def clean_string(string):
    """
    Cleans an input string by replacing all newlines with spaces, and then
    removing all letters not in *allowable_letters*
    """
    string = string.replace("\n"," ")
    return "".join([i for i in string.lower().strip() if i in allowable_letters])

def load_bigrams(text):
    """
    Takes a string which has already been cleaned, and returns a dictionary
    of conditional bigram probabilities
    
    cond_bigram[(a,b)] = P(X_{n+1} = b | X_{n} = a)
    
    Uses Laplace smoothing with size $1$, to remove zero transition probabilities
    """
    bigram_counter = Counter(list(it.product(allowable_letters, repeat=2)))
    gram_counter = Counter(allowable_letters*len(allowable_letters))
    for l1, l2 in zip(text,text[1:]):
        bigram_counter[(l1, l2)] += 1
        gram_counter[l1] += 1
    cond_bigram = {k:v/gram_counter[k[0]] for k,v in bigram_counter.items()}
    return cond_bigram
    
def bigram_from_file(filename):
    """
    Given a filename, this reads it, cleans it, and returns the conditional bigram
    """
    file_text = open(filename).read()
    file_text = clean_string(file_text)
    return load_bigrams(file_text)

def reverse_cipher(cipher):
    return {v:k for k, v in cipher.items()}

def print_differences(cipher1, cipher2):
    for k in cipher1:
        if cipher1[k] != cipher2[k]:
            print("%s: %s  %s"%(k,cipher1[k], cipher2[k]))
            
def num_errors(cipher, encoded_text, original_text):
    decoded = np.array(list(decode_text(encoded_text, cipher)))
    original = np.array(list(original_text))
    num_errors = np.count_nonzero(decoded != original)
    return num_errors

def get_secret_text(student_id):
    with open('Lab_04_data/secret_strings.p', 'rb') as f:
        texts = pickle.load(f)
    return texts[student_id % len(texts)]

# Lab 4: Code Breaking by MCMC #

## Identify Your Lab Partner ##

This is a multiple choice question. Please select **ONE** of following options that best describes how you complete this lab.

- I am doing this lab by myself and I don't have a partner.
- My partner for this lab is [PARTNER'S NAME] with email [berkeley.edu email address]. [SUBMITTER'S NAME] will submit to Gradescope and add the other partner to the group on Gradescope after submission.

Please copy and paste **ONE** of above statements and fill in blanks if needed. If you work with a partner, make sure only one of you submit on Gradescope and that the other member of the group is added to the submission on Gradescope. Refer to the bottom of the notebook for submission instructions.


**Type your answer in this cell.**

\newpage

## Breaking a Substitution Code ##

Cryptography is the study of algorithms used to encode and decode messages. Markov Chain Monte Carlo (MCMC) methods have been successfully used to decode messages encrypted using substitution codes and also [more complex](http://probability.ca/jeff/ftpdir/decipherart.pdf) encryption methods. In this lab you will apply MCMC and the Metropolis algorithm to decode English text that has been encrypted with a substitution code.

The lab is based on the paper [The Markov Chain Monte Carlo Revolution](https://math.uchicago.edu/~shmuel/Network-course-readings/MCMCRev.pdf) by [Persi Diaconis](https://en.wikipedia.org/wiki/Persi_Diaconis). It was presented at the 25th anniversary celebrations of MSRI up the hill, and appeared in the Bulletin of the American Mathematical Society in November 2008. The code is based on [Simulation and Solving Substitution Codes](http://www-users.york.ac.uk/~sbc502/decode.pdf) written by [Stephen Connor](https://www.york.ac.uk/maths/staff/stephen-connor/) in 2003.

Recall that in class we worked with a tiny alphabet and a short message that had been encoded using a substitution code. We used a Markov chain model and a scoring system to choose the decoder that had the highest score. That is, we chose the decoder that had the highest likelihood given the message. 

Because the alphabet was small, we were able to list all the decoders and their corresponding scores. When the alphabet is large, making an exhaustive list of all possible decoders and scores is not feasible. That's where the Monte Carlo part of the algorithm comes in. The idea is to choose decoders at random using an algorithm that favors good decoders over bad ones.

In this lab you will work with an alphabet consisting of all 26 English letters as well as a space character that will have a special status. **Before you begin, please review Sections [11.2](http://prob140.org/textbook/content/Chapter_11/02_Code_Breaking.html) (Code Breaking) and [11.3](http://prob140.org/textbook/content/Chapter_11/03_Metropolis_Algorithm.html) (Metropolis Algorithm) of the textbook.** The lab follows those sections closely.

Unlike all the other labs, this lab is not divided into parts. It's just one small project. You will learn how to:
- Apply a decoder to encoded text
- Use transformations for improved numerical accuracy
- Implement the Metropolis algorithm 
- Decode a message encrypted by a substitution code

The programming needed for this lab is more complex than what can reasonably be expected based on just Data 8 background, so much of it has been done for you. Those of you who have more extensive Python knowledge might be interested in looking at the details.

Please start by running all the cells above this one.

## Instructions

Similar to your homeworks, your labs will generally have two components: a written portion and a portion that also involves code.  
- Written work should be completed on paper, and coding questions should be done in the notebook.
- Start the work for the written portions of each section on a new page.
- You are welcome to $\LaTeX$ your answers to the written portions, but staff will not be able to assist you with $\LaTeX$ related issues. 
- Show your work. Give reasoning. The question isn't always going to ask for it, because we assume that you will provide justification for your answers. Every answer should contain a calculation, reasoning, or diagrams that are clearly labeled to show what's going on.
- It is your responsibility to ensure that both components of the lab are submitted completely and properly to Gradescope. **Make sure to assign each page of your pdf to the correct question.** 
- **Refer to the bottom of the notebook for submission instructions.**

## Markov Chain on Characters: Alphabet and Bigrams ##
The text that you are going to decode was written in English. For data on the frequencies of all the different bigrams in English, we will start by counting all the bigrams in *War and Peace* by Tolstoy. That is one of the longest novels in English (actually in Russian, but we are using an English translation from Project Gutenberg) and is often used as a "corpus" or body of language on which to base analyses of other text.

To keep the calculations manageable, we will restrict ourselves to a 27-character alphabet, consisting of the 26 lower case letters and a space. The cell below places all 27 in a list.

In [ ]:
allowable_letters = list("abcdefghijklmnopqrstuvwxyz ")

In the cell below, we find the relative frequencies of all bigrams in War and Peace and place them in the dictionary `wp_bigrams`. You don't need to know what a dictionary is. It's fine to imagine it containing the same information as a Table that has a column with the bigrams and another column with the relative frequencies. 

In [ ]:
wp_bigrams = bigram_from_file("Lab_04_data/warandpeace.txt")

### 1. Transition Matrix ###
As in class, we model the sequence of characters in the text as a Markov chain. This model seems overly simplified, but it turns out to be enough for many purposes such as the one in this lab.

The state space of this chain is the alphabet of 27 characters. Of course the text has other punctuation, but we are stripping all of it for simplicity. We are also replacing upper case letters with lower case ones.

Construct the transition matrix `wp_bigram_mc` for this Markov chain based on the relative frequencies in `wp_bigrams` defined above. Refer to [Section 10.1.5](http://prob140.org/textbook/content/Chapter_10/01_Transitions.html#sticky-reflecting-random-walk) for an example that uses relevant code.

We have started the code off for you by defining the transition function `wp_transition`. Use `allowable_letters` in your definition.

In [ ]:
def wp_transition(a, b):
    if (a, b) in wp_bigrams:
        return wp_bigrams[(a, b)]
    return 0

wp_bigram_mc = ...
wp_bigram_mc

## Decoders ##

As you know, when a substitution code has been used for encryption, a decoder is a permutation of the alphabet. We apply this permutation to our encoded text in order to generate the decoded text.

We will be representing the decoder as a Python dictionary. Here is an example of how a dictionary works, using a three-letter alphabet 'a', 'b', 'c'.

If the decoder is ['b', 'c', 'a'] then the substitutions for decoding the encrypted message are

$$
a \to b ~~~~~~~~ b \to c ~~~~~~~~ c \to a
$$

This decoder written as a dictionary looks like this:

    simple_decoder = {'a':'b','b':'c','c':a'}

To access a value in the decoder, use the method `.get`. For example, to access the translation of *a*, use
    
    simple_decoder.get('a')

In [ ]:
simple_decoder = {'a':'b', 'b':'c', 'c':'a'}
simple_decoder.get('a')

## Keeping Spaces Intact ##
To simplify calculations, we are assuming that our substitution code keeps spaces unchanged. Letters can precede and follow spaces, but spaces will be fixed points in the encoding permutation. 

So our decoder must keep spaces unchanged as well. The following cell defines a function called `random_decoder` which will randomly generate a decoder. It will operate on the 26 letters, omitting the space.

Run the cell below. The function `random_decoder` constructs a random permutation of the alphabet and places it in a dictionary to make it a decoder.

In [ ]:
# We don't operate on spaces
decoder_letters = np.array(list("abcdefghijklmnopqrstuvwxyz")) 

# Set up a dictionary with one entry for each letter
identity_decoder = {letter:letter for letter in decoder_letters}

# Random starting decoder
def random_decoder():
    """ Random decoder """
    new_letters = decoder_letters.copy()
    np.random.shuffle(new_letters)
    return {orig:new for orig ,new in zip(decoder_letters, new_letters)}

\newpage

### 2. Applying a Decoder ###

Define a function `decode_text` that takes as arguments a `string` to be decoded and the `decoder`. It should return the decoded string.

Remember that we are only decoding alphabetical characters. If a character is not in the decoder (e.g. a space), leave the character alone.

In [ ]:
def decode_text(string, decoder):
    new_string = ""
    
    for char in string:
        if char in decoder:
            new_letter = ...
        else:
            new_letter = ...
        
        # Now we append the letter to the back of the new string
        new_string = new_string + new_letter
        
    return new_string

Run the cell below to check that your function works as expected.

In [ ]:
simple_decoder = {'a':'b', 'b':'c', 'c':'a'}
decode_text("abcba", simple_decoder)

## Score of a Decoder ##

For letters $x_1$ and $x_2$, let $B(x_1, x_2)$ be the $(x_1, x_2)$th entry in the bigram transition matrix. Suppose the decoded text consists of the characters $x_0, x_1, \ldots , x_n$. Recall from class that we defined the *score* of the corresponding decoder $i$ to be

$$
s(i) = \prod_{k=0}^{n-1}B_i(x_k, x_{k+1})
$$

Here's a tweak to what was done in class. When the string is long, the score of any decoder is going to be very small. This causes problems with numerical accuracy. So we will work with the log-score instead, that is, with the $\log$ of the score. 

$$
\log(s(i)) = \sum_{k=0}^{n-1} \log(B_i(x_k, x_{k+1}))
$$


\newpage

### 3. Log Probability of Path ###

Define a function `log_prob_path` that takes two arguments:
- a string `message` that is a sequence of letters 
- the bigram transition matrix `mc` of a `MarkovChain` 

and returns the log of the probability of the path `message` taken by `mc`, given that the first character of `message` is the initial state of the path.

Though `mc.prob_of_path` returns the probability of the path, don't use `np.log(mc.prob_of_path)`. Each path has very small probability, so the numbers will round too soon and your computations might be inaccurate.

Instead, you can use the function [`mc.log_prob_of_path`](http://prob140.org/prob140/_autosummary/prob140.MarkovChain.log_prob_of_path.html#prob140.MarkovChain.log_prob_of_path). This takes as its first argument the starting character of the string, and as its next argument a list or array containing the characters in the rest of the string. It returns the log of the probability of the path, conditional on the starting character.

Some string methods will be helpful:

* `string[0]` returns the first letter of a string. For example:
```python
>>> x = 'HELLO'
>>> x[0]
'H'
```
* `string[1:]` returns everything except the first letter of a string. For example:
```python
>>> x[1:]
'ELLO'
```
* `list(string)` splits the string into a list of its characters. This was used above to generate `allowable_letters`.

In [ ]:
def log_prob_path(string, bigram_mc):
    start = ...
    
    path = list(...)
    
    return bigram_mc.log_prob_of_path(start, path)

\newpage

### 4. Log Score of Decoder ###

Define a function `log_score` that takes the following arguments:

- an encrypted string `string`
- a decoder `decoder`
- a bigram transition matrix `bigrams`

The function should decode the encrypted string using the decoder, and return the log score of the decoder.

Use the functions `decode_text` and `log_prob_path` that you wrote earlier.

In [ ]:
def log_score(string, decoder, bigrams):
    ...
    return ...

## Markov Transitions on Decoders: A Proposal Matrix ##
The Metropolis algorithm we will use is essentially the same as the one defined in class and proved to converge to the correct distribution. There are tweaks to increase computatational accuracy and to deal with spaces, but you shouldn't worry about either of them.

Please have Section [11.3](http://prob140.org/textbook/content/Chapter_11/03_Metropolis_Algorithm.html) of the textbook at hand as you go through the exercises below.

Our goal is to find the "best" decoder, that is, the decoder that has the highest score. Since we can't list all the decoders, we are going to try out decoders at random – but not uniformly at random. So we need a rule for moving from one decoder to another.

The Metropolis algorithm starts by creating an essentially arbitrary transition matrix on decoders. Given that we have just tried out decoder $i$, we will pick another decoder to try out, using a probability distribution that just satisfies a couple of simple conditions. Think of this as a possibly dumb starting point. We'll use it as a base to create a smarter process.

You will now define the starting point, which is a transition matrix $Q$ on the space of all decoders, satisfying some conditions needed for the Metropolis algorithm. The purpose of $Q$ is to propose a decoder $j$ for us to try out next, given that we have just tried out decoder $i$. We will still have to decide whether or not to accept that proposal, but we'll get to that later. At this stage, we will simply define $Q$.

- Define $Q$ as follows: given that the chain is at decoder $i$, pick another decoder by randomly swapping two elements of $i$. For any two decoders $i$ and $j$, let

\begin{equation}
Q(i, j) = 
 \begin{cases}
 \frac{1}{\binom{26}{2}} & \text{if } i \text{ and } j \text{ differ in exactly two places} \\
0 & \text{otherwise}
 \end{cases}
\end{equation}

Two decoders differ in exactly two places when two letters have "switched" positions. For example, in a 3 letter alphabet the decoders `abc` and `bac` differ in exactly two places because `a` and `b` have "switched" positions.

You will show below that $Q$ satisfies the conditions for the proposal transition matrix used in the Metropolis algorithm.

\newpage

### 5a) Properties of $Q$ ###
$Q$ is a huge matrix. How many rows does it have? For reference, `special.comb(n, k)` evaluates to $\binom{n}{k}$ and `special.factorial(n, exact=True)` evaluates to $n!$. The modules were imported at the start of the lab.

In [ ]:
...

Define the three decoders (that is, permutations) below:

- decoder_1: alphabetical order, that is, `abcdef ... xyz`
- decoder_2: `bacdef ... xyz`, that is, `ba` followed by the other letters in alphabetical order
- decoder_3: `badcef ... xyz`, that is, `badc` followed by the other letters in alphabetical order

For each of (i)-(iv) below, type 1/(26_choose_2) if you think the value is $1/\binom{26}{2}$ and 0 otherwise, and enter the reason for your choice.


(i) Q(decoder_1, decoder_2)  =  

because 

(ii) Q(decoder_2, decoder_1) =  

because

(iii) Q(decoder_1, decoder_3) = 

because

(iv) Q(decoder_2, decoder_3) = 

because

**Symmetric Transition Matrix:** Explain why $Q$ is a symmetric transition matrix. 

[Note: You have to establish that $Q$ is symmetric and that it is a transition matrix. For the latter, it is a good idea to focus on the row corresponding to decoder_1 above. What criteria do the entries of each row have to satisfy? Why are the criteria satisfied?]


**Your answer here.**

**Irreducible:** It is a fact that Q is the transition matrix of an irreducible chain, but formally establishing that takes a bit of writing. To avoid complicated writing, let's reduce the problem to an alphabet of just three letters `a`, `b`, and `c` and transition behavior defined analogously on the six decoders, with positive transition probabilities each equal to 1/(3_choose_2) = 1/3 if the two permutations differ in exactly two places. **Why does this create an irreducible chain?**


**Your answer here.**

For any size of alphabet, there is a [famous algorithm](https://en.wikipedia.org/wiki/Steinhaus%E2%80%93Johnson%E2%80%93Trotter_algorithm) for creating all possible permutations by successively swapping two *adjacent* elements. It was known to bell-ringers in 17th-century England. Adjacent swaps are among the transitions allowed by the matrix $Q$, so the chain is irreducible. 

**Proposal Chain:** Now explain why $Q$ satisfies all the criteria for the transition matrix of a proposal chain. You have done all the work already; carefully read [Section 11.3.1](http://prob140.org/textbook/content/Chapter_11/03_Metropolis_Algorithm.html#id1) to see what is needed.


**Your answer here.**

### 5b) Making a Proposal to Move ###

Define the function `generate_proposed_decoder` which takes an existing decoder and returns a new decoder as follows: it randomly picks two different letters of the alphabet and swaps the decoding for those letters. Make sure to use `decoder_letters` rather than `allowable_letters` because we want to keep the spaces as is.

For example, if we have the decoder `{'a':'b','b':'c','c':a'}` and decide to swap `a` and `b`, our new decoder should be `{'a':'c','b':'b','c':a'}`. 

There are several different ways you can use Python to draw random samples. In this instance we recommend `np.random.choice`. It takes two arguments:
- a list from which to sample uniformly at random with replacement
- the number of times to sample

The optional argument `replace=False` leads to simple random sampling.

It returns an array consisting of the sampled elements.

In [ ]:
def generate_proposed_decoder(decoder):
    new_decoder = decoder.copy()
    
    letters = np.random.choice(
        ..., size = 2, ...)
    
    letter1 = ...
    letter2 = ...
    
    new_value_of_letter1 = ...
    new_value_of_letter2 = ...

    # This code replaces the value of letter1 and letter2 
    # with new_value_of_letter1 and new_value_of_letter2
    new_decoder[letter1] = new_value_of_letter1
    new_decoder[letter2] = new_value_of_letter2
    return new_decoder

## Markov Chain on Decoders: The Metropolis Algorithm ##

Remember that we are looking for the decoder that has the highest score. So, given that we have just tried out one decoder, we would like to move on to not just any other random decoder but a *better* decoder. That is, we'd like a process that is likely to move us to a decoder with a higher score.

The proposal matrix $Q$ won't do this, because $Q$ doesn't involve the scores at all. The Metropolis algorithm uses $Q$ *as well as the scores* to help us reach our destination. You will now implement the details.

Imagine a probability distribution $\pi$ with entries proportional to the scores. This distribution can be thought of as consisting of the likelihood of each decoder given the text. Suppose we can create a Markov chain in which Decoder $i$ appears among our randomly selected decoders with target probability $\pi(i)$ in the long run. Then the decoder we are most likely to see in the long run is the one with the highest likelihood.

The trouble is that there are too many decoders for us to list, so we can't normalize all the scores to get a probability distribution. But we don't actually need the entire distribution $\pi$. We are just looking for the mode of $\pi$. 

### Acceptance Ratios ###
For this, we are going to use the fact that for any two decoders $i$ and $j$, we can easily see which one has the higher score. One way to capture this information is to calculate the ratio

$$
r(i, j) = \frac{\pi(j)}{\pi(i)} = \frac{s(j)}{s(i)}
$$

**The Metropolis algorithm starts with the proposal transition matrix $Q$ and uses these ratios to create a new chain that has our target $\pi$ as its limit distribution.** 

Recall that if the new chain is at $i$, then the decision about whether or not to accept a transition to a new state $j$ depends on comparing the ratio $r(i, j)$ to the value $1$.

**Required Reading at this point:** [Section 11.3.2](http://prob140.org/textbook/content/Chapter_11/03_Metropolis_Algorithm.html#how-to-think-about-the-algorithm) which discusses how to use the ratio $r$ to decide whether or not to accept a proposal made by $Q$.

#### Log Ratio of Scores ####

Because we are working with log scores and not the scores themselves, every statement that we made in class about scores has to be rewritten in terms of log scores. For example, the ratio can be written as

$$
\frac{s(j)}{s(i)} = e^{\log\left(\frac{s(j)}{s(i)}\right)} = e^{\log(s(j)) - \log(s(i))}
$$

You will have to calculate the ratios, so keep in mind that `np.e**(x)` evaluates to $e^x$.

Write the condition $r(i, j) < 1$ in terms of $\log(s(i))$ and $\log(s(j))$. Justify your answer.


**Your answer here.**

## Implementing the Metropolis Algorithm ##

\newpage

### 6. An Encoded Message ###

Enter your student id in the cell below to access your secret encoded message. By the end of the lab, you should be able to decode your string.

In [ ]:
student_id = ...
secret_text = get_secret_text(student_id)
secret_text

\newpage

### 7. Implementation ###

Define the function `metropolis` which will take as its arguments:

- the encrypted text, as a string `string_to_decode`
- the transition matrix of bigrams `bigrams`
- the number of repetitions for running the Metropolis algorithm `reps`

The function should return the decoder that it arrives at after performing the Metropolis algorithm the specified number of times. 

If the number of repetitions is large, you know from class that the distribution of this random decoder is close to the desired distribution $\pi$ which is the likelihood distribution of the decoder given the data. Therefore you expect to end up with a decoder that has a high likelihood compared to the other decoders.

Between iterations, you should keep track of the following variables:

1. `best_decoder`: the decoder that has the highest log score
2. `best_score`: the log score associated with the best_decoder
3. `decoder`: the decoder that you are currently working with
4. `last_score`: the log score of the current decoder

In each iteration, you should do the following:

1. Generate a new decoder based on the current decoder; the "new" decoder might be the same as the current one.
2. Calculate the log score of your new decoder.
3. **Important:** Follow the Metropolis algorithm to decide whether or not to move to a new decoder.
4. If the new decoder's log score is the best seen so far, update `best_score` and `best_decoder`

In [ ]:
import time

def p_coin(p):
    """
    Flips a coin that comes up heads with probability p
    and returns 1 if Heads, 0 if Tails
    """
    return np.random.random() < p


def metropolis(string_to_decode, bigrams, reps):
    decoder = random_decoder() # Starting decoder
    
    best_decoder = decoder
    best_score = log_score(
        string_to_decode,
        best_decoder,
        bigrams
    )
    
    last_score = best_score
    
    for rep in np.arange(reps):
        
        # This will print out our progress
        if rep*10%reps == 0: # Repeat every 10%
            time.sleep(.01)
            decoded_text = decode_text(
                string_to_decode,
                best_decoder
            )[:40]
            print('Score: %.00f \t Guess: %s'%(best_score, decoded_text))
            
        #########################
        # Your code starts here #
        #########################
        proposed_decoder = ...
        log_s_orig = ...
        log_s_new = ...
        
        # If better than before or p-coin flip works
        if log_s_new ... log_s_orig or p_coin(...): 
            
            ...
            
            if log_s_new > best_score:
                ...
    return best_decoder

\newpage

## Running It ##

The text you are going to first encrypt and then decode is taken from [Winston Churchill's speech](https://en.wikipedia.org/wiki/We_shall_fight_on_the_beaches) to the House of Commons on 4 June 1940. Naturally, it contains upper case letters and punctuation that we have not included in our alphabet. In the cell below, the function `clean_string` strips out all the punctuation and replaces upper case letters by the corresponding lower case letters.

In [ ]:
ogtext = """
I have, myself, full confidence that if all do their duty, if nothing is neglected,
and if the best arrangements are made, as they are being made, we shall prove ourselves 
once again able to defend our Island home, to ride out the storm of war, and to outlive 
the menace of tyranny, if necessary for years, if necessary alone. At any rate, that is
what we are going to try to do. That is the resolve of His Majesty’s Government-every man
of them. That is the will of Parliament and the nation.
"""
original_string = clean_string(ogtext)
encoding_cipher = random_decoder()
string_to_decode = decode_text(original_string,encoding_cipher)
string_to_decode

### 8a) Finding the Decoder ###
Run the following cell to generate your decoder. You may have to run the cell a couple of times to get the correct decoder.

In [ ]:
new_decoder = metropolis(string_to_decode, wp_bigram_mc, 10000) 

Run the cell below to decode the entire string.

In [ ]:
decode_text(string_to_decode, new_decoder)

### 8b) Decoding Your Secret Message

`secret_text` contains your customized, secret message. Use MCMC to decode it. (Note: Even with correct implementation of previous parts, it may take a few runs to successfully decode the text due to randomness in the algorithm. Even then, it may fail to properly decode some letters, especially those which generally don't occur very frequently in English, such as $q$ or $j$.)

In [ ]:
secret_decoder = metropolis(...)
decode_text(secret_text, secret_decoder)

## Conclusion ##
What you have learned:
- How to implement the Metropolis algorithm to decode text encrypted by a substitution code, with simplifying assumptions about spaces, capitalization, and punctuation
- MCMC works
- Abstract concepts such as balance and convergence of $n$-step transition matrices are remarkable in their own right and also have powerful applications

## Submission Instructions ##

Many assignments throughout the course will have a written portion and a code portion. Please follow the directions below to properly submit both portions.

### Written Portion ###
*  Scan all the pages into a PDF. You can use any scanner or a phone using applications such as CamScanner. Please **DO NOT** simply take pictures using your phone. 
* Please start a new page for each question. If you have already written multiple questions on the same page, you can crop the image in CamScanner or fold your page over (the old-fashioned way). This helps expedite grading.
* It is your responsibility to check that all the work on all the scanned pages is legible.
* If you used $\LaTeX$ to do the written portions, you do not need to do any scanning; you can just download the whole notebook as a PDF via LaTeX.

### Code Portion ###
* Save your notebook using `File > Save and Checkpoint`.
* Generate a PDF file using `File > Download As > PDF via LaTeX`. This might take a few seconds and will automatically download a PDF version of this notebook.
    * If you have issues, please post a follow-up on the general Lab 4 Ed thread.
    
### Submitting ###
* Combine the PDFs from the written and code portions into one PDF. [Here](https://smallpdf.com/merge-pdf) is a useful tool for doing so. 
* Submit the assignment to Lab 4 on Gradescope. 
* **Make sure to assign each page of your pdf to the correct question.**
* **It is your responsibility to verify that all of your work shows up in your final PDF submission.**

If you are having difficulties scanning, uploading, or submitting your work, please read the [Ed Thread](https://edstem.org/us/courses/35049/discussion/2398718) on this topic and post a follow-up on the general Lab 4 Ed thread.

## **We will not grade assignments which do not have pages selected for each question.** ##